<a href="https://colab.research.google.com/github/ujjwalr-y20cs161/AI-lab/blob/main/DecisionTreeV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import math

In [6]:
#read the data set

data = pd.read_csv("play_tennis.csv")
data.head(14)

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def Entropy(data):
  p,n = 0,0

  for i in data.index:
    if(data.loc[i]["play"] ==  "Yes"):p+=1
    elif(data.loc[i]["play"] == "No"):n+=1

  total = p+n

  if p == 0 or n == 0 : return 0

  p,n = p/total,n/total

  return -p*math.log(p,2)-n*math.log(n,2)

def EntropyAttr(data,attr):
  ent = []

  if attr not in data.columns :
    print("Invalid Attribute")
    return

  values = list(set(data[attr]))
  # print(values)
  n = len(data)

  for value in values:
    # print(value)
    pos= 0
    neg= 0

    for i in data.index:
      # print(data[attr][i],data["play"][i] ==  "Yes",value)
      if(data.loc[i][attr] == value and data.loc[i]["play"] ==  "Yes" ):pos+=1
      elif(data.loc[i][attr] == value and data.loc[i]["play"] == "No"):neg+=1
      # else: print(f"Invalid attribute value in {attr} column")
    totV = pos+neg
    if(pos == 0 or neg == 0) : continue
    # print(attr,value,totV,pos,neg)
    pos,neg = pos/totV,neg/totV
    # print((-pos*math.log(pos,2) - neg*math.log(neg,2)))

    ent.append((totV/n)*(-pos*math.log(pos,2) - neg*math.log(neg,2)))
  return sum(ent)

def InfoGain(data,attr):
  return Entropy(data) - EntropyAttr(data,attr)


In [9]:
class Node:
    def __init__(self):
        self.children=[]
        self.value=""
        self.isLeaf=False
        self.pred=""

In [10]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new example", new," is:", child.pred)
                exit
            else:
                classify (child.children[0], new)

In [15]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
        #print ("\n",examples)
        gain = InfoGain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    #print ("\nMax feature attr",max_feat)
    uniq = np.unique(examples[max_feat])
    #print ("\n",uniq)
    for u in uniq:
        #print ("\n",u)
        subdata = examples[examples[max_feat] == u]
        #print ("\n",subdata)
        if Entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["play"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            subdata.reset_index().drop('index',axis=1)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root



In [21]:
def decisionTree(data,attrs):

  root = Node()

  maxGain = 0
  maxFeature = ""

  for attr in attrs:
    gain = InfoGain(data,attr)
    print(gain,attr)
    if gain > maxGain :
      maxGain = gain
      maxFeature = attr
    print(maxGain,maxFeature)
  root.value = maxFeature
  # print(maxFeature,data[maxFeature])

  # uniq = np.unique(data[maxFeature])

  # print(uniq)


In [13]:
attrs = data.columns.tolist()
attrs.remove('day')
attrs.remove('play')
print(attrs)

['outlook', 'temp', 'humidity', 'wind']


In [18]:
printTree(ID3(data,attrs))

outlook
	Overcast ->  ['Yes']

	Rain
		wind
			Strong ->  ['No']

			Weak ->  ['Yes']

	Sunny
		humidity
			High ->  ['No']

			Normal ->  ['Yes']



In [17]:
for i in data.index:
  print(data.loc[i]["play"])
  print("***************************")

No
***************************
No
***************************
Yes
***************************
Yes
***************************
Yes
***************************
No
***************************
Yes
***************************
No
***************************
Yes
***************************
Yes
***************************
Yes
***************************
Yes
***************************
Yes
***************************
No
***************************


In [22]:
decisionTree(data,attrs)

0.2467498197744391 outlook
0.2467498197744391 outlook
0.029222565658954647 temp
0.2467498197744391 outlook
0.15183550136234136 humidity
0.2467498197744391 outlook
0.04812703040826927 wind
0.2467498197744391 outlook
